In [1]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: devscripts 2.22.1ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of devscripts or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


#### First we need to configure the any pyspark application with SparkConf() method

In [2]:
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

In [3]:
conf = SparkConf().setAppName("Pyspark Cofiguration demo").setMaster("local[2]")
#spark = SparkSession.builder.appName("FirstSparksession").getOrCreate()
print(conf.get("spark.master"))
print(conf.get("spark.app.name"))

local[2]
Pyspark Cofiguration demo


In [4]:
sc = SparkConf()

In [5]:
pip install findspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import findspark
findspark.init()

In [7]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession

In [1]:
# SparkSession.builder.appName("First").getOrCreate()

In [2]:
import pandas as pd
import numpy as np

In [32]:
di = {"date": ["01-01-2015","02-01-2015","03-01-2015","06-01-2015",
        "07-01-2015","11-01-2015","12-01-2015","13-01-2015"]}

In [33]:
df = pd.DataFrame(di)
df.dtypes

date    object
dtype: object

In [51]:
df1 = pd.DataFrame(pd.to_datetime(df["date"], dayfirst=True, format = "%d-%m-%Y"))
df1.dtypes

date    datetime64[ns]
dtype: object

In [52]:
df1.groupby(["date"]).size()

date
2015-01-01    1
2015-01-02    1
2015-01-03    1
2015-01-06    1
2015-01-07    1
2015-01-11    1
2015-01-12    1
2015-01-13    1
dtype: int64

In [53]:
!pip install scipy

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: devscripts 2.22.1ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of devscripts or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [54]:
from scipy.stats import rankdata

In [55]:
rankdata(df1)

array([1., 2., 3., 4., 5., 6., 7., 8.])

In [59]:
df1['rank'] = range(1, len(df1.sort_values(by = "date"))+1)
df1

,date,rank
0,2015-01-01,1
1,2015-01-02,2
2,2015-01-03,3
3,2015-01-06,4
4,2015-01-07,5
5,2015-01-11,6
6,2015-01-12,7
7,2015-01-13,8


In [112]:
import pandas as pd

# Given DataFrame
di = {"date": ["01-01-2015","02-01-2015","03-01-2015","06-01-2015",
               "07-01-2015","11-01-2015","12-01-2015","13-01-2015"]}

df = pd.DataFrame(di)

# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')

df

,date
0,2015-01-01
1,2015-01-02
2,2015-01-03
3,2015-01-06
4,2015-01-07
5,2015-01-11
6,2015-01-12
7,2015-01-13


In [113]:
df["day"] = df["date"].dt.day
df

,date,day
0,2015-01-01,1
1,2015-01-02,2
2,2015-01-03,3
3,2015-01-06,6
4,2015-01-07,7
5,2015-01-11,11
6,2015-01-12,12
7,2015-01-13,13


In [114]:
df

,date,day
0,2015-01-01,1
1,2015-01-02,2
2,2015-01-03,3
3,2015-01-06,6
4,2015-01-07,7
5,2015-01-11,11
6,2015-01-12,12
7,2015-01-13,13


In [119]:
t = (df["date"]!=df["date"].shift()).cumsum()
t

0    1
1    2
2    3
3    4
4    5
5    6
6    7
7    8
Name: date, dtype: int64

In [120]:
df.groupby(t)['date'].cumcount()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
dtype: int64

In [78]:
# Identify the gaps and reset the rank to 1 for the new range
df['rank'] = (df['date'].shift())

# Display the DataFrame with consecutive ranks
print(df)


        date       rank diff_rank
0 2015-01-01        NaT       NaT
1 2015-01-02 2015-01-01    1 days
2 2015-01-03 2015-01-02    1 days
3 2015-01-06 2015-01-03    3 days
4 2015-01-07 2015-01-06    1 days
5 2015-01-11 2015-01-07    4 days
6 2015-01-12 2015-01-11    1 days
7 2015-01-13 2015-01-12    1 days


### To fill te missing values with last Non-NULL categories

In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.types import * 
from functools import reduce
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder.appName("myapp").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/19 15:46:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [36]:
data =[('chocolates','5-star')
,(None,'dairy milk')
,(None,'perk')
,(None,'eclair')
,('Biscuits','britannia')
,(None,'good day')
,(None,'boost')]

In [37]:
df = spark.createDataFrame(data)
df

DataFrame[_1: string, _2: string]

In [38]:
df = df.toDF("category" , "brand_name")
print(df)

DataFrame[category: string, brand_name: string]


In [39]:
df = df.withColumn("order_id" , monotonically_increasing_id())
print(df)

DataFrame[category: string, brand_name: string, order_id: bigint]


In [40]:
w_forward = Window.partitionBy().orderBy("order_id"). \
rowsBetween(Window.unboundedPreceding,Window.currentRow) 

print(w_forward)

In [41]:
df.withColumn('category',last('category',ignorenulls=True). \
              over(w_forward)).drop("order_id").show()

24/01/19 13:54:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/19 13:54:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/19 13:54:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+----------+
|  category|brand_name|
+----------+----------+
|chocolates|    5-star|
|chocolates|dairy milk|
|chocolates|      perk|
|chocolates|    eclair|
|  Biscuits| britannia|
|  Biscuits|  good day|
|  Biscuits|     boost|
+----------+----------+



24/01/19 13:54:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/19 13:54:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [42]:
spark.stop()

### Calculate the % of marks for each student along with class

In [79]:
stu_df = [(1,'Steve'),(2,'David'),(3,'Aryan'), (4,'Nandu')]
marks_df = [(1,'pyspark',90),(1,'sql',100),
            (2,'sql',70),(2,'pyspark',60),
            (3,'sql',60),(3,'pyspark',55),
            (4,'sql',27),(4,'pyspark',30)]

In [80]:
stu_df, marks_df

([(1, 'Steve'), (2, 'David'), (3, 'Aryan'), (4, 'Nandu')],
 [(1, 'pyspark', 90),
  (1, 'sql', 100),
  (2, 'sql', 70),
  (2, 'pyspark', 60),
  (3, 'sql', 60),
  (3, 'pyspark', 55),
  (4, 'sql', 27),
  (4, 'pyspark', 30)])

In [81]:
stud_data = spark.createDataFrame(stu_df).toDF("student_id" , "name")
stud_data.show()

+----------+-----+
|student_id| name|
+----------+-----+
|         1|Steve|
|         2|David|
|         3|Aryan|
|         4|Nandu|
+----------+-----+



In [82]:
stud_data.printSchema()

root
 |-- student_id: long (nullable = true)
 |-- name: string (nullable = true)



In [83]:
marks_data = spark.createDataFrame(marks_df).toDF("student_id" , "subject_name", "marks")
marks_data.show()

+----------+------------+-----+
|student_id|subject_name|marks|
+----------+------------+-----+
|         1|     pyspark|   90|
|         1|         sql|  100|
|         2|         sql|   70|
|         2|     pyspark|   60|
|         3|         sql|   60|
|         3|     pyspark|   55|
|         4|         sql|   27|
|         4|     pyspark|   30|
+----------+------------+-----+



In [84]:
marks_data.printSchema()

root
 |-- student_id: long (nullable = true)
 |-- subject_name: string (nullable = true)
 |-- marks: long (nullable = true)



In [85]:
stud_data.unionByName(marks_data,allowMissingColumns = True).show()

+----------+-----+------------+-----+
|student_id| name|subject_name|marks|
+----------+-----+------------+-----+
|         1|Steve|        null| null|
|         2|David|        null| null|
|         3|Aryan|        null| null|
|         4|Nandu|        null| null|
|         1| null|     pyspark|   90|
|         1| null|         sql|  100|
|         2| null|         sql|   70|
|         2| null|     pyspark|   60|
|         3| null|         sql|   60|
|         3| null|     pyspark|   55|
|         4| null|         sql|   27|
|         4| null|     pyspark|   30|
+----------+-----+------------+-----+



In [86]:
joined_df = stud_data.join(marks_data, on = "student_id")
joined_df.show()

+----------+-----+------------+-----+
|student_id| name|subject_name|marks|
+----------+-----+------------+-----+
|         1|Steve|     pyspark|   90|
|         1|Steve|         sql|  100|
|         2|David|         sql|   70|
|         2|David|     pyspark|   60|
|         3|Aryan|         sql|   60|
|         3|Aryan|     pyspark|   55|
|         4|Nandu|         sql|   27|
|         4|Nandu|     pyspark|   30|
+----------+-----+------------+-----+



In [87]:
group_data = joined_df.groupBy(["student_id","name"]).avg()
drop_unknkwn_group_data = group_data.drop("avg(student_id)")
drop_unknkwn_group_data.show()

+----------+-----+----------+
|student_id| name|avg(marks)|
+----------+-----+----------+
|         1|Steve|      95.0|
|         2|David|      65.0|
|         3|Aryan|      57.5|
|         4|Nandu|      28.5|
+----------+-----+----------+



In [88]:
drop_unknkwn_group_data.select(col("avg(marks)").alias("Percentage"))

DataFrame[Percentage: double]

In [89]:
percent_data = drop_unknkwn_group_data.withColumnRenamed("avg(marks)", "Percentage")
percent_data.show()

+----------+-----+----------+
|student_id| name|Percentage|
+----------+-----+----------+
|         1|Steve|      95.0|
|         2|David|      65.0|
|         3|Aryan|      57.5|
|         4|Nandu|      28.5|
+----------+-----+----------+



In [90]:
percent_data.select("name").show()

+-----+
| name|
+-----+
|Steve|
|David|
|Aryan|
|Nandu|
+-----+



In [91]:
percent_data.where(col("Percentage")>70).show()

+----------+-----+----------+
|student_id| name|Percentage|
+----------+-----+----------+
|         1|Steve|      95.0|
+----------+-----+----------+



In [92]:
percent_data.withColumn("Result", when((col("Percentage") >= 70), "Distinction"). \
                        when((col("Percentage") >= 60) & (col("Percentage") <= 69), "First Class"). \
                        when((col("Percentage") >= 50) & (col("Percentage") <= 59), "Second Class"). \
                        when((col("Percentage") >= 40) & (col("Percentage") <= 49), "Third Class"). \
                        otherwise("Fail")).show()

+----------+-----+----------+------------+
|student_id| name|Percentage|      Result|
+----------+-----+----------+------------+
|         1|Steve|      95.0| Distinction|
|         2|David|      65.0| First Class|
|         3|Aryan|      57.5|Second Class|
|         4|Nandu|      28.5|        Fail|
+----------+-----+----------+------------+

